In [7]:
from pathlib import Path
from math import sqrt
import polars as pl
from skrub import TableReport

dataset = "ACSEmployment_binarized"
# base_model = "skrub_logistic"
base_model = "skrub"
audit_budget = 1_000
# n_repetitions = 15
n_repetitions = 5
# n_repetitions = 1
entropy = 12345678
tnr = 1.0
tpr = 1.0
output = Path(f"../generated/stealthiness{n_repetitions}_{dataset}_{base_model}.jsonl")

params = [
    "dataset",
    "model_name",
    # "model_params",
    "strategy",
    "strategy_params",
    "audit_budget",
    "detection_tpr",
    "detection_tnr",
    # "entropy",
]
records = pl.read_ndjson(output)

In [9]:
TableReport(records.select(pl.all().exclude("strategy_params")))

Processing column  18 / 18


,,,,,,,,,,,,,,,,,


In [4]:
(
    records.select(pl.all().exclude("model_params"))
    .with_columns(auditset_hamming=1 - pl.col("utility_audit"))
    .group_by(params)
    .agg(
        pl.all().mean(),
        pl.all().std().name.suffix("_std"),
    )
    .with_columns(
        pl.col("demographic_parity_audit_std") / sqrt(n_repetitions),
        pl.col("auditset_hamming_std") / sqrt(n_repetitions),
        pl.col("manipulation_hamming_std") / sqrt(n_repetitions),
        pl.col("strategy_params").struct.json_encode(),
    )
    .sort("strategy")
)

DuplicateError: column with name 'entropy_std' has more than one occurrence

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'group_by' <---
DF ["dataset", "model_name", "strategy", "strategy_params"]; PROJECT */31 COLUMNS; SELECTION: None